In [40]:
import json
import requests
import numpy as np
import pandas as pd
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

In [2]:
aws_rdb = DatabaseConnector()
aws_rdb.list_db_tables()

['legacy_store_details', 'legacy_users', 'orders_table']

In [5]:
orders_df = DataExtractor.read_rds_table('orders_table')
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120123 entries, 0 to 120122
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   level_0           120123 non-null  int64  
 1   date_uuid         120123 non-null  object 
 2   first_name        15284 non-null   object 
 3   last_name         15284 non-null   object 
 4   user_uuid         120123 non-null  object 
 5   card_number       120123 non-null  int64  
 6   store_code        120123 non-null  object 
 7   product_code      120123 non-null  object 
 8   1                 0 non-null       float64
 9   product_quantity  120123 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 10.1+ MB


In [7]:
orders_df.drop(columns=['first_name', 'last_name', '1', 'level_0'], inplace=True)
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120123 entries, 0 to 120122
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   date_uuid         120123 non-null  object
 1   user_uuid         120123 non-null  object
 2   card_number       120123 non-null  int64 
 3   store_code        120123 non-null  object
 4   product_code      120123 non-null  object
 5   product_quantity  120123 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 6.4+ MB


In [18]:
orders_df['store_code'] = orders_df['store_code'].astype('category')

In [23]:
response = requests.get('https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json')
response

<Response [200]>

In [30]:
events_df = pd.DataFrame(response.json())

In [31]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120161 entries, 0 to 120160
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   timestamp    120161 non-null  object
 1   month        120161 non-null  object
 2   year         120161 non-null  object
 3   day          120161 non-null  object
 4   time_period  120161 non-null  object
 5   date_uuid    120161 non-null  object
dtypes: object(6)
memory usage: 6.4+ MB


In [32]:
events_df.head(10)

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
5,02:03:25,8,2002,2,Late_Hours,cea14ebe-06b2-4cee-8394-972101419375
6,20:41:58,1,1993,14,Evening,5b72f016-349c-4649-aea0-bcc4fd1b9ee6
7,22:36:19,3,2006,15,Evening,5d598d23-ee71-4fc9-a895-77fd3891a7c4
8,18:06:15,12,1994,14,Evening,ad50dbd6-b736-4ffe-a745-6ebed29e4cdf
9,16:12:22,11,1994,1,Midday,92fc92a3-c4a0-49a4-a2ee-40a37f9d11dc


In [39]:
events_df.loc[events_df['time_period'] == 'NULL']

,timestamp,month,year,day,time_period,date_uuid
11873,NULL,NULL,NULL,NULL,NULL,NULL
14280,NULL,NULL,NULL,NULL,NULL,NULL
20358,NULL,NULL,NULL,NULL,NULL,NULL
23525,NULL,NULL,NULL,NULL,NULL,NULL
23794,NULL,NULL,NULL,NULL,NULL,NULL
27347,NULL,NULL,NULL,NULL,NULL,NULL
35767,NULL,NULL,NULL,NULL,NULL,NULL
50988,NULL,NULL,NULL,NULL,NULL,NULL
55040,NULL,NULL,NULL,NULL,NULL,NULL
67893,NULL,NULL,NULL,NULL,NULL,NULL


In [45]:
events_df.replace('NULL', np.nan, inplace=True)
events_df.loc[events_df['time_period'] == 'NULL']

,timestamp,month,year,day,time_period,date_uuid


In [50]:
events_df.dropna(how='all', inplace=True)
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120146 entries, 0 to 120160
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   timestamp    120146 non-null  object
 1   month        120146 non-null  object
 2   year         120146 non-null  object
 3   day          120146 non-null  object
 4   time_period  120146 non-null  object
 5   date_uuid    120146 non-null  object
dtypes: object(6)
memory usage: 6.4+ MB


In [52]:
events_df.loc[events_df['time_period'] == '5MUU1NKRED']

,timestamp,month,year,day,time_period,date_uuid
53213,OIOLKX0XI4,2VZEREEIKB,FXC3K5LZZX,1OFIGIX6Q9,5MUU1NKRED,I9GYGGEMVT


In [54]:
# events_df['temp_time'] = pd.to_datetime(events_df['timestamp'], errors='coerce').dt.time

In [61]:
# events_df.loc[events_df['temp_time'].isna()]
# events_df = events_df.loc[events_df['temp_time'].notna()]

,timestamp,month,year,day,time_period,date_uuid,temp_time,dt_string,datetime
7580,SAAZHF87TI,1YMRDJNU2T,FTKRTQHFZE,7EPFWYOELT,DXBU6GX1VC,3A21WYQSY7,NaT,FTKRTQHFZE 1YMRDJNU2T 7EPFWYOELT SAAZHF87TI,NaT
8344,75E4ECDVH6,9GN4VIO5A8,14NRQ80L5E,K4702YOYPT,OEOXBP8X6D,66ULRXEWSU,NaT,14NRQ80L5E 9GN4VIO5A8 K4702YOYPT 75E4ECDVH6,NaT
13873,JUVMW8TKUC,NF46JOZMTA,33F45BZPSP,ZS0PDDW72O,1Z18F4RM05,XXHOTW6WA7,NaT,33F45BZPSP NF46JOZMTA ZS0PDDW72O JUVMW8TKUC,NaT
27444,J8CSDZCCRZ,LZLLPZ0ZUA,G3DEZY8UW6,TTH9JE93YZ,GT3JKF575H,3TL8P43R9J,NaT,G3DEZY8UW6 LZLLPZ0ZUA TTH9JE93YZ J8CSDZCCRZ,NaT
28521,MMC1FBD3XH,YULO5U0ZAM,0M8BGI0CI3,CZ35WI1011,CM5MTJKXMH,ZAJPIRYLAL,NaT,0M8BGI0CI3 YULO5U0ZAM CZ35WI1011 MMC1FBD3XH,NaT
32353,AND6XEV4U3,SAT4V9O2DL,O17F6WE1TD,PYXMXY268K,5OQGE7K2AV,RKDX2XBIJ4,NaT,O17F6WE1TD SAT4V9O2DL PYXMXY268K AND6XEV4U3,NaT
39870,MD3ZLFQ12Y,3ZZ5UCZR5D,9DKC6PW41E,XAOROIDDK6,1JCRGU3GIE,10NCUUENK0,NaT,9DKC6PW41E 3ZZ5UCZR5D XAOROIDDK6 MD3ZLFQ12Y,NaT
46709,MLX6W1MJ8G,DGQAH7M1HQ,I5367BRUVN,DEQME0YBTK,SQX52VSNMM,3CEP13OI0F,NaT,I5367BRUVN DGQAH7M1HQ DEQME0YBTK MLX6W1MJ8G,NaT
48592,WR4RL0H97H,4FHLELF101,EB2N507OZ0,DK93ZX02KL,ALOGCWS9Y3,IW5ZTTFD6Z,NaT,EB2N507OZ0 4FHLELF101 DK93ZX02KL WR4RL0H97H,NaT
52129,YO6TX1S7CS,22JSMNGJCU,KO7BGRPOKH,8XA0GSY2Z6,7DNU2UWFP7,AHXVM4Q3LH,NaT,KO7BGRPOKH 22JSMNGJCU 8XA0GSY2Z6 YO6TX1S7CS,NaT


In [58]:
events_df['dt_string'] = events_df['year'] + ' ' + events_df['month'] + ' ' + events_df['day'] + ' ' + events_df['timestamp']
events_df['datetime'] = pd.to_datetime(events_df['dt_string'], errors='coerce')

In [60]:
events_df.tail(10)

,timestamp,month,year,day,time_period,date_uuid,temp_time,dt_string,datetime
120151,18:17:21,3,1998,14,Evening,83132084-a7a3-49ba-bb7a-8e61711dfb8e,18:17:21,1998 3 14 18:17:21,1998-03-14 18:17:21
120152,04:10:40,11,2009,19,Late_Hours,ec8164e6-ad93-40fc-97a6-6fae2a5803c5,04:10:40,2009 11 19 04:10:40,2009-11-19 04:10:40
120153,15:20:00,8,2000,11,Midday,d35d0cfa-6bec-4519-b681-e0b2ab2a98e4,15:20:00,2000 8 11 15:20:00,2000-08-11 15:20:00
120154,22:56:25,5,2003,29,Evening,95823602-7c7d-48c3-80ff-cbe40d5f0a21,22:56:25,2003 5 29 22:56:25,2003-05-29 22:56:25
120155,20:22:24,6,2006,30,Evening,afc0dce1-f08f-4276-968c-58ecedf8f48d,20:22:24,2006 6 30 20:22:24,2006-06-30 20:22:24
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7,22:56:56,2022 11 12 22:56:56,2022-11-12 22:56:56
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6,18:25:20,1997 5 31 18:25:20,1997-05-31 18:25:20
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e,18:21:40,2011 9 13 18:21:40,2011-09-13 18:21:40
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb,19:10:53,2013 7 12 19:10:53,2013-07-12 19:10:53
120160,21:17:12,3,2008,18,Evening,55c228c7-14ee-4d46-99a9-01dfc57d1adf,21:17:12,2008 3 18 21:17:12,2008-03-18 21:17:12


In [62]:
events_df = events_df.loc[events_df['datetime'].notna()]

In [63]:
events_df.loc[events_df['datetime'].isna()]

,timestamp,month,year,day,time_period,date_uuid,temp_time,dt_string,datetime


In [72]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120123 entries, 0 to 120160
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   timestamp    120123 non-null  object        
 1   month        120123 non-null  int64         
 2   year         120123 non-null  int64         
 3   day          120123 non-null  int64         
 4   time_period  120123 non-null  category      
 5   date_uuid    120123 non-null  object        
 6   temp_time    120123 non-null  object        
 7   datetime     120123 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3), object(3)
memory usage: 7.4+ MB


In [65]:
events_df.drop(columns='dt_string', inplace=True)

In [ ]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp']).dt.time

In [71]:
events_df['month'] = events_df['month'].astype('int64')
events_df['year'] = events_df['year'].astype('int64')
events_df['day'] = events_df['day'].astype('int64')
events_df['time_period'] = events_df['time_period'].astype('category')